# Working with obstacles
This notebook aims at giving the basis to work with obstacles, first by examplifying the basic API of HPP-FCL, then by writing an optimization problem under noncollision avoidance constraint. One of the main message is that the collision constraint is hard, and that a lot of work remains necessary, to make it efficient in practice.

We first show how to compute the minimal ellipsoid that encapsulate a body of the robot. 
Then a inverse-geometry problem is written under the constraint that a set of points remains outside of the encapsulating ellipsoid.

The notebook is written for a simple UR10 robot, and you should be able to mix it with notebook #3 to make a complete trajectory optimization for a humanoid.



In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will use the following tools:
- the ur10 model and the Talos model (loaded by example-robot-data)
- HPP-FCL, through the Pinocchio API
- pinocchio.casadi for writing the problem and computing its derivatives
- the IpOpt solver wrapped in casadi
- the meshcat viewer

In [3]:
# %load tp4/generated/encapsulating_ellipses_import
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
from utils.meshcat_viewer_wrapper import MeshcatVisualizer


We will load the UR10 model, but feel free to change it for any model you like.

In [5]:
# %load tp4/generated/encapsulating_ellipses_load
# --- Load robot model
robot = robex.load('ur10')
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)


Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [6]:
viz.viewer.jupyter_cell()

## Accessing the vertices of the collisions objects
Let's quickly summarize the basics of collision avoidance in Pinocchio. Our models are most often loaded from URDF storage, which typically contains two sets of 3D objects: the visuals, very detailed and often colored meshed used for display; and the collisions, less detailed objects used for a fair computation of the collisions. Both are stored in the robot_wrapper, under collision_model and visual_model.

In [7]:
print(robot.collision_model,robot.visual_model)


Nb geometry objects = 8
Name: 	 
base_link_0
Parent frame ID: 	 
4
Parent joint ID: 	 
0
Position in parent frame: 	 
  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0

Absolute path to mesh file: 	 
/opt/openrobots/share/example-robot-data/robots/../../example-robot-data/robots/ur_description/meshes/ur10/collision/base.stl
Scale for transformation of the mesh: 	 
1 1 1
Disable collision: 	 
0


Name: 	 
shoulder_link_0
Parent frame ID: 	 
8
Parent joint ID: 	 
1
Position in parent frame: 	 
  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0

Absolute path to mesh file: 	 
/opt/openrobots/share/example-robot-data/robots/../../example-robot-data/robots/ur_description/meshes/ur10/collision/shoulder.stl
Scale for transformation of the mesh: 	 
1 1 1
Disable collision: 	 
0


Name: 	 
upper_arm_link_0
Parent frame ID: 	 
10
Parent joint ID: 	 
2
Position in parent frame: 	 
  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0

Absolute path to mesh file: 	 
/opt/openrobots/share/example-robot-data/robots/../../example-robot-data/robot

Both are corresponding to the same structure of type pin.GeometryModel. It mostly contains a list of geometry objects (that we are going to use) and a list of collision pair (very useful, but not in this notebook).


In [11]:
geom=robot.collision_model.geometryObjects[0]

A geometry object is a 3d shape attached to a parent joint at a specific placement. It mostly contains 3 fields: geom.parentJoint the index of the parent joint ; geom.placement, the placement of the 3d shape in the frame of the parent joint; and geom.geometry, a specific structure representing the shape.

In [13]:
shape = geom.geometry
print(shape)

Shapes can be simple features (sphere, capsule, ellipsoid, box, etc), or soup of polygon. This is the most frequent case when loading a model from URDF. Then we are mostly interested by the list of vertices:

In [37]:
vertices = shape.vertices()
print(vertices)

[[ 0.06140732 -0.07101219  0.        ]
 [ 0.0107041  -0.0919913   0.0290448 ]
 [ 0.0101903  -0.09200149  0.00837344]
 [ 0.02182996 -0.07544291  0.03799992]
 [ 0.00950617 -0.09175908  0.02993541]
 [-0.05599856  0.07227855  0.        ]
 [-0.00221407  0.0750876   0.03639787]
 [-0.00294936  0.07494199  0.        ]
 [ 0.07494199 -0.00294936  0.        ]
 [ 0.07494199  0.00294936  0.        ]
 [ 0.07498645  0.00142002  0.03799992]
 [ 0.07227855  0.05599856  0.        ]
 [ 0.07222586  0.05659395  0.01199996]
 [ 0.07186049  0.05875337  0.        ]
 [ 0.07227855 -0.05599856  0.01199996]
 [ 0.07439428 -0.00950783  0.03799992]
 [ 0.07186049 -0.05875337  0.01199996]
 [ 0.05875337 -0.07186049  0.        ]
 [ 0.06140732 -0.07101219  0.01199996]
 [-0.01993542 -0.07599997  0.03799992]
 [ 0.01993542 -0.07599997  0.03799992]
 [-0.01004421 -0.09199476  0.02970439]
 [-0.02092278 -0.07585775  0.03799992]
 [ 0.04303449 -0.06142485  0.03799992]
 [ 0.0661509  -0.06812119  0.01199996]
 [ 0.04697597 -0.05846536

Most of the time, when you are working with HPP-FCL through Pinocchio, you will not go that deep in the collision description, but rather access it through high level functions of HPP-FCL, like computing distances, detecting collisions, etc. Here we are going to use this list to compute the minimal encapsulating ellispoid.

## Encapsulating

### Problem formulation
An ellispoid $\mathcal{E}$ can be defined by a symmetric positive matrix $A$ and a center 3d vector $c$, as the set of points respecting:
$$ \forall p \in \mathcal{E}, (p-c)^T A (p-c) \le 1$$

The matrix $A$ can be deduced from the orientation of the main axes $R$ and the 3 radii $r=(r_1,r_2,r_3)$ by:
$$A=R \ \textrm{diag}(\frac{1}{r^2}) R^T$$
where $\textrm{diag}(\frac{1}{r^2})$ is the diagonal matrix formed with the inverted squared radii.

Inside an optimization program, $R$ is more conveniently represented by ... (if you thought Euler angles, please leave the room) ... angle vectors or quaternion. We will you the first with $R\triangleq exp(w)$ and $w\in R^3$ and unconstrained 3d angle vector.

The problem can then be written:

Decide: 
- $w\in R^3$ the ellipsoid orientation
- $c \in R^3$ the ellipsoid center
- $r \in R^3$ the radii

Minimizing: the ellipsoid volum $\prod_{i=1}^{3} r_i$

Subject to:
- $r>=0$
- $\for v \in V:  (v-c)^T A (v-c) \le 1$
with $V$ the list of vertices, and $A$ defined as above mentionned from the decision variables.



### Helpers 
When writing this problem, $A$ and $c$ will be represented in the same frame as $V$. Remember that the vertices of the shape are given in the shape frame. Converting them in the joint frame can be done with geom.placement:

In [38]:
geom.placement.act(vertices[0,:])

array([ 0.06140732, -0.07101219,  0.        ])

We will need the following simple Casadi helper to get rid of the SX/MX syntax.

In [39]:
# %load tp4/generated/encapsulating_ellipses_helper
cw = casadi.SX.sym('w',3)
exp = casadi.Function('exp3',[cw], [cpin.exp3(cw)])


Now write an optimization problem with Casadi to compute the representation of the minimal encapsulating ellipsoid in the frame of the parent joint. For that, follow the steps:

1. Define the decision variables $w,c,r$.

In [40]:
# %load tp4/generated/encapsulating_ellipses_vars
opti = casadi.Opti()
var_w = opti.variable(3)
var_r = opti.variable(3)
var_c = opti.variable(3)


2. Shape the matrix $A$ from the decision variables.

In [41]:
# %load tp4/generated/encapsulating_ellipses_RA
# The ellipsoid matrix is represented by w=log3(R),diag(P) with R,P=eig(A)
R = exp(var_w)
A = R@casadi.diag(1/var_r**2)@R.T


3. Define the cost.

In [42]:
# %load tp4/generated/encapsulating_ellipses_cost
totalcost = var_r[0]*var_r[1]*var_r[2]


4. Define the radius positivity.

In [43]:
# %load tp4/generated/encapsulating_ellipses_rplus
opti.subject_to( var_r >= 0)


5. Define the encapsulating constraint.

In [46]:
# %load tp4/generated/encapsulating_ellipses_points
for g_v in vertices:
    # g_v is the vertex v expressed in the geometry frame.
    # Convert point from geometry frame to joint frame
    j_v = geom.placement.act(g_v)
    # Constraint the ellipsoid to be including the point
    opti.subject_to( (j_v-var_c).T@A@(j_v-var_c)  <= 1  )


6. Solve and recover the optimal values.

In [48]:
# %load tp4/generated/encapsulating_ellipses_solve
opti.minimize(totalcost)
opti.solver("ipopt") # set numerical backend
opti.set_initial(var_r,10)

sol = opti.solve_limited()

sol_r = opti.value(var_r)
sol_A = opti.value(A)
sol_c = opti.value(var_c)
sol_R = opti.value(exp(var_w))



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     1668
Number of nonzeros in Lagrangian Hessian.............:       45

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

### Display and future use

The ellipoid can be displayed in Meshcat. You need to give the radii to create the shape, and then place it at the center with proper orientaion. So Meshcat needs $(c,r,R)$ and not $(A,c)$. You can get $r,R$ from $A$ by Eigen decomposion.

In [52]:
# %load tp4/generated/encapsulating_ellipses_meshcat
# Build the ellipsoid 3d shape
# Ellipsoid in meshcat
viz.addEllipsoid('el',sol_r,[.3,.9,.3,.3])
# jMel is the placement of the ellipsoid in the joint frame
jMel = pin.SE3(sol_R,sol_c)


We can now place the initial shape geom, the vertices represented by small spheres, and the ellispoid, at a consistent location.

In [56]:
# %load tp4/generated/encapsulating_ellipses_vizplace
# Place the body, the vertices and the ellispod at a random configuration oMj_rand
oMj_rand = pin.SE3.Random()
viz.applyConfiguration(viz.getViewerNodeName(geom,pin.VISUAL),oMj_rand)
for i in  np.arange(0,vertices.shape[0]):
    viz.applyConfiguration(f'world/point_{i}',
                           oMj_rand.act(vertices[i]).tolist()+[1,0,0,0])
viz.applyConfiguration('el',oMj_rand*jMel)


## Inverse geometry with collisions